In [1]:
import nltk

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml



# データ情報

In [2]:
from nltk.corpus.util import LazyCorpusLoader
from nltk.corpus.reader import *

# Loading the corpus
ma_reuters = LazyCorpusLoader(
    'ma_reuters', CategorizedPlaintextCorpusReader, '(training|test).*',
    cat_file='cats.txt', encoding='ISO-8859-2')

# Load MA_Reuters
documents = ma_reuters.fileids()
print (str(len(documents)) + " total articles")
# extracting training and testing data (document ID)
train_docs_id = list(filter(lambda doc: doc.startswith("train"), documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"), documents))
print (str(len(train_docs_id)) + " training data")
print (str(len(test_docs_id)) + " testing data")
# Training and testing data
train_docs = [ma_reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [ma_reuters.raw(doc_id) for doc_id in test_docs_id]
 
# print the total number of categories
categories = ma_reuters.categories()
num_categories = len(categories)
print (num_categories, " categories")
print (categories)

10700 total articles
7713 training data
2987 testing data
55  categories
['acq', 'alum', 'barley', 'bop', 'carcass', 'cocoa', 'coffee', 'copper', 'corn', 'cotton', 'cpi', 'crude', 'dlr', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'hog', 'housing', 'interest', 'ipi', 'iron-steel', 'jobs', 'lead', 'livestock', 'meal-feed', 'money-fx', 'money-supply', 'nat-gas', 'oilseed', 'orange', 'palm-oil', 'pet-chem', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


# ココアカテゴリの文書

In [26]:
# Documents in a category
category_docs = ma_reuters.fileids("cocoa");
document_id = category_docs[0] # The first document
# print the inside document
print (ma_reuters.raw(document_id))

COCOA EXPORTERS EXPECTED TO LIMIT SALES
  Major cocoa exporters are likely to
  limit sales in the weeks ahead in an effort to boost world
  prices, sources close to a meeting of the Cocoa Producers
  Alliance (CPA) said.
      The sources said the depressed world market had been one of
  the main topics discussed in a closed door meeting of the
  11-member CPA which began on Monday.
      They said producers agreed that cutting sales would aid the
  buffer stock manager of a new international cocoa pact in his
  effort to support prices.
      Major cocoa producing and consuming nations agreed
  operation rules for the buffer stock at a meeting in London
  last month and the stock manager is expected to enter the
  market soon.
      Prices, under the weight of three successive cocoa
  surpluses, recently fell to the level at which the manager has
  to buy cocoa under stock rules.
      The buffer stock aims to keep prices within a pre-set range
  by buying when prices fall and sellin

# TF-IF

In [24]:
from nltk import word_tokenize
import re # regular expression
 
def tokenize(text): # returning tokens
    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(text))

    p = re.compile('[a-zA-Z]+')
    filtered_tokens = list(filter (lambda token: p.match(token) and len(token) >= min_length, words))
    return filtered_tokens

from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english', tokenizer=tokenize)
# fit_transform
vectorised_train_documents = vectorizer.fit_transform(train_docs)
# transform
vectorised_test_documents = vectorizer.transform(test_docs)
print("converted to TF-IF model")
print("training document dimension ：",vectorised_train_documents.shape)
print("testing document dimension：",vectorised_test_documents.shape)

converted to TF-IF model
training document dimension ： (7713, 26985)
testing document dimension： (2987, 26985)



# Score

In [25]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([ma_reuters.categories(doc_id) for doc_id in train_docs_id])
test_labels = mlb.transform([ma_reuters.categories(doc_id) for doc_id in test_docs_id])

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

# multi-class, multi-label classification and prediction
OVR_classifier = OneVsRestClassifier(LinearSVC(random_state=41)) 
OVR_classifier.fit(vectorised_train_documents, train_labels)
OVR_predictions = OVR_classifier.predict(vectorised_test_documents)

import numpy as np
# Jaccard coefficient
from sklearn.metrics import jaccard_score
print ("Jaccard coef:",np.round(jaccard_score(test_labels, OVR_predictions, average='samples'),3))

# Hamming Loss
from sklearn.metrics import hamming_loss
print ("Hamming Loss:",np.round(hamming_loss(test_labels, OVR_predictions),3))

Jaccard coef: 0.86
Hamming Loss: 0.005


# 課題2-2

In [11]:
print ("Jaccard coef:",np.round(jaccard_score(test_labels, OVR_predictions, average=None),3))
print("Max:")
print(categories[np.argmax(jaccard_score(test_labels, OVR_predictions, average=None))])
print("Min:")
score = jaccard_score(test_labels, OVR_predictions, average=None)
mini = tmp.min()
for i in range(len(tmp)):
    if mini==tmp[i]:
        print(categories[i])

Jaccard coef: [0.94  0.435 0.643 0.625 0.421 0.944 0.931 0.833 0.724 0.55  0.414 0.81
 0.653 0.969 0.2   0.471 0.75  0.625 0.836 0.333 0.75  0.621 0.833 0.556
 0.524 0.    0.407 0.316 0.651 0.703 0.471 0.5   0.727 0.6   0.    0.444
 0.579 0.5   0.36  0.75  0.66  0.125 0.273 0.231 0.    0.436 0.    0.757
 0.667 0.672 0.421 0.697 0.5   0.    0.462]
Max:
earn
Min:
lead
pet-chem
soy-oil
strategic-metal
yen
